In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
import os
import torchvision.transforms as transforms
import torch.nn as nn
from torch.utils.data import DataLoader,Dataset
from PIL import Image

### Defining Transforms

In [ ]:
transform1 = transforms.Compose([transforms.Resize((128,128)) ,transforms.ToTensor()])
transform2 = transforms.Compose([transforms.Resize((128,128)), transforms.RandomVerticalFlip(p=1) , transforms.ToTensor()])
transform3 = transforms.Compose([transforms.Resize((128,128)), transforms.RandomHorizontalFlip(p=1) , transforms.ToTensor()])

### Function to load all image from a directory

In [ ]:
def load_img(dirs , transform):
    
    y = []
    for file in os.listdir(dirs):
        img_name = os.path.join(dirs, file)
        image = Image.open(img_name)
        y.append(transform(image))
    return y           

### Defining the Model

In [ ]:
class CNN(nn.Module):
    def __init__(self , out1 , out2 , out3):
        super(CNN,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels = 3 , out_channels = out1 , kernel_size = 3 , padding = 0)
        self.activate1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2)
        
        self.cnn2 = nn.Conv2d(in_channels = out1 , out_channels = out2 , kernel_size = 3 , stride = 1 , padding = 0)
        self.activate2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size = 2 )
        
        self.cnn3 = nn.Conv2d(in_channels = out2 , out_channels = out3 , kernel_size = 3 , stride = 1 , padding = 0)
        self.activate3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size = 2 )
        
        self.fc1 = nn.Linear(out3*14*14 , 2)
        
        self.sigmoid = nn.Sigmoid()
        
    
    def forward(self , x):
        
        x = self.cnn1(x)
        x = self.activate1(x)
        x = self.maxpool1(x)
    
        x = self.cnn2(x)
        x = self.activate2(x)
        x = self.maxpool2(x)
        
        x = self.cnn3(x)
        x = self.activate3(x)
        x = self.maxpool3(x)
        
        x = x.view(x.size(0) , -1)
        x = self.fc1(x)
        x = self.sigmoid(x)
        
        return x

### Defining three datasets - one Normal and two Augmented
##### Of the two augmented datsets , one has images flipped vertically and the other horizontally

In [ ]:
Xtrain1 = load_img('F:/Machine Learning/Cats Vs Dogs/train/train' , transform1)
ytrain1 = np.zeros((25000,2))
for i in range(12500):
    ytrain1[i,0] = 1
    ytrain1[i+12500,1] = 1

In [ ]:
Xtrain2 = load_img('F:/Machine Learning/Cats Vs Dogs/train/train' , transform2)
ytrain2 = np.zeros((25000,2))
for i in range(12500):
    ytrain2[i,0] = 1
    ytrain2[i+12500,1] = 1

In [ ]:
Xtrain3 = load_img('F:/Machine Learning/Cats Vs Dogs/train/train' , transform3)
ytrain3 = np.zeros((25000,2))
for i in range(12500):
    ytrain3[i,0] = 1
    ytrain3[i+12500,1] = 1

#### Plot a random image and label

In [ ]:
plt.imshow(Xtrain2[134][1].numpy())
print(ytrain2[134])

### Combine Data and labels to create a DataLoader Object

In [ ]:
train_data = []
for i in range(len(Xtrain1)):
    train_data.append([Xtrain1[i], ytrain1[i]])
    train_data.append([Xtrain2[i], ytrain2[i]])
    train_data.append([Xtrain3[i], ytrain3[i]])

In [ ]:
trainloader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=100)

### Create a model , define loss criterion and the optimizer

In [ ]:
model = CNN(32,32,64)
criterion = nn.BCELoss()
optimizer = torch.optim.RMSprop(model.parameters() , lr = 0.01 , momentum = 0.9)

### Function to plot accuracy and loss

In [ ]:
def plot_accuracy_loss(accuracy , LOSS):
    plt.plot(accuracy)
    plt.title("Training Accuracy")
    plt.xlabel("Steps")
    plt.ylabel("Accuracy")
    plt.show()
    
    plt.plot(LOSS , color = 'red')
    plt.title("Training Loss")
    plt.xlabel("Steps")
    plt.ylabel("Loss")
    plt.show()

### Run the Program

In [ ]:
import time

In [ ]:
epochs = 1
accuracy = []
for e in range(epochs):
    print("Epoch :" , e+1)
    temp = []
    LOSS = []
    i==0
    now = time.time()
    for x , y in trainloader:
        i = i+1
        y = y.type(torch.FloatTensor)
        _,y_red = y.max(1)
        z = model(x)
        
        loss = criterion(z , y)
        LOSS.append(loss)
        _,yhat = z.max(1)
        if i==1:
            print(yhat , y_red)
        temp.append((yhat==y_red).sum())
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    accuracy.append(np.mean(temp))
    end = time.time()
    print("Time : " ,end-now)
    plot_accuracy_loss(temp , LOSS)

### Deallocate Memory for Xtrain

In [ ]:
del Xtrain1,Xtrain2,Xtrain3

### Load Testing Data

In [ ]:
Xtest = load_img('F:/Machine Learning/Cats Vs Dogs/test/test' , transform1)

### Obatin Predictions

In [ ]:
testloader = torch.utils.data.DataLoader(Xtest , batch_size = 100)

In [ ]:
y_predict = []
for x in testloader:
    z = model(x)
    _,yhat = z.max(1)
    y_predict.append(yhat) 

In [ ]:
print(y_predict)

### Write the output to a CSV file

In [ ]:
pd.DataFrame(np.array(y_predict)).to_csv("F:/predict.csv")